# Initialization

In [1]:
import logging

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'png'
%config InlineBackend.figure_format = 'retina'

# Загрузка данных

In [3]:
items = pd.read_parquet('goodsread/items.parquet')
events = pd.read_parquet('goodsread/events.parquet')

# Разбиение с учётом хронологии

Рекомендательные системы на практике работают с учётом хронологии. Поэтому поток событий для тренировки и валидации полезно делить на то, что уже случилось, и что ещё случится. Это позволяет проводить валидацию на тех же пользователях, на которых тренировались, но на их событиях в будущем.

# === Знакомство: "холодный" старт

In [4]:
events.iloc[0].to_dict()

{'user_id': 1229132,
 'book_id': 22034,
 'started_at': datetime.date(2015, 7, 12),
 'read_at': datetime.date(2015, 7, 17),
 'is_read': True,
 'rating': 5,
 'is_reviewed': False,
 'started_at_month': datetime.date(2015, 7, 1)}

Разбиение выборки (задание 2 из 6)

In [5]:
# зададим точку разбиения
train_test_global_time_split_date = pd.to_datetime('2017-08-01').date()

train_test_global_time_split_idx = (
    events["started_at"] < train_test_global_time_split_date
)

events_train = events[train_test_global_time_split_idx]
events_test = events[~train_test_global_time_split_idx]

# количество пользователей в train и test
users_train = events_train['user_id'].drop_duplicates()
users_test = events_test['user_id'].drop_duplicates()

# количество пользователей, которые есть и в train, и в test
common_users = set(users_train) & set(users_test)

print(len(users_train), len(users_test), len(common_users))

428220 123223 120858


Идентифицируем холодных пользователей (задание 3 из 6)

In [6]:
cold_users = set(users_test) - common_users

print(len(cold_users))

2365


Топ-100 наиболее популярных книг с 2015 года и со средней оценкой не меньше 4 (задание 4 из 6)

In [7]:
top_pop_start_date = pd.to_datetime('2015-01-01').date()

item_popularity = (
    events_train
    .query('started_at >= @top_pop_start_date')
    .groupby(['book_id'])
    .agg(users=('user_id', 'nunique'), avg_rating=('rating', 'mean'))
    .reset_index()
)
item_popularity['popularity_weighted'] = (
    item_popularity['users'] * item_popularity['avg_rating']
)

In [8]:
item_popularity.head(3)

,book_id,users,avg_rating,popularity_weighted
0,1,8728,4.796059,41860.0
1,2,9528,4.685873,44647.0
2,3,15139,4.706057,71245.0


In [9]:
# сортируем по убыванию взвешенной популярности
item_popularity = item_popularity.sort_values(
    'popularity_weighted',
    ascending=False
)
# выбираем первые 100 айтемов со средней оценкой avg_rating не меньше 4
top_k_pop_items = (
    item_popularity
    .query('avg_rating >= 4')
    .sort_values('popularity_weighted', ascending=False)
    .head(100)
)

top_k_pop_items.head(3)

,book_id,users,avg_rating,popularity_weighted
32387,18007564,20207,4.321275,87320.0
32623,18143977,19462,4.290669,83505.0
30695,16096824,16770,4.301014,72128.0


In [10]:
# добавляем информацию о книгах
if not set(top_k_pop_items.columns) & {
    "author", "title", "genre_and_votes", "publication_year"
}:
    top_k_pop_items = top_k_pop_items.merge(
        items.set_index("book_id")[
            ["author", "title", "genre_and_votes", "publication_year"]
        ],
        on="book_id"
    )

# with pd.option_context('display.max_rows', 100):
#     display(top_k_pop_items[["book_id", "author", "title", "publication_year", "users", "avg_rating", "popularity_weighted", "genre_and_votes"]]) 

In [11]:
top_k_pop_items.head(3)

,book_id,users,avg_rating,popularity_weighted,author,title,genre_and_votes,publication_year
0,18007564,20207,4.321275,87320.0,Andy Weir,The Martian,"{'Science Fiction': 11966, 'Fiction': 8430}",2014
1,18143977,19462,4.290669,83505.0,Anthony Doerr,All the Light We Cannot See,"{'Historical-Historical Fiction': 13679, 'Fict...",2014
2,16096824,16770,4.301014,72128.0,Sarah J. Maas,A Court of Thorns and Roses (A Court of Thorns...,"{'Fantasy': 14326, 'Young Adult': 4662, 'Roman...",2015


Задание 5 из 6

In [12]:
# Для событий холодных пользователей добавляем данные из top100 (если есть)
cold_users_events_with_recs = (
    events_test[events_test['user_id'].isin(cold_users)]
    .merge(top_k_pop_items, on='book_id', how="left")
)

cold_users_events_with_recs.head(3)

,user_id,book_id,started_at,read_at,is_read,rating,is_reviewed,started_at_month,users,avg_rating,popularity_weighted,author,title,genre_and_votes,publication_year
0,1361610,6900,2017-10-09,2017-10-13,True,4,False,2017-10-01,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
1,1361610,12555,2017-09-21,2017-10-11,True,3,False,2017-09-01,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
2,1361610,25899336,2017-09-12,2017-09-17,True,4,True,2017-09-01,4798.0,4.427261,21242.0,Paul Kalanithi,When Breath Becomes Air,"{'Nonfiction': 5848, 'Autobiography-Memoir': 3...",<NA>


In [13]:
# Индексы событий НЕ из top100
cold_user_items_no_avg_rating_idx = (
    cold_users_events_with_recs['avg_rating'].isnull()
)
# События холодных пользователей с книгами из top100
cold_user_recs = (
    cold_users_events_with_recs[~cold_user_items_no_avg_rating_idx]
    [['user_id', 'book_id', 'rating', 'avg_rating']]
)
cold_user_recs.head(3)

,user_id,book_id,rating,avg_rating
2,1361610,25899336,4,4.427261
5,1338996,16096824,5,4.301014
8,1338996,18692431,5,4.071454


In [14]:
len(cold_user_recs)

1912

In [15]:
len(cold_users_events_with_recs)

9672

In [16]:
cold_user_items_no_avg_rating_idx.sum()

7760

In [17]:
1 - len(cold_user_recs) / len(cold_users_events_with_recs)

0.8023159636062862

Задание 6 из 6

In [18]:
# посчитаем метрики рекомендаций
from sklearn.metrics import mean_squared_error, mean_absolute_error

rmse = mean_squared_error(
    cold_user_recs['rating'], 
    cold_user_recs['avg_rating'],
    squared=False
)
mae = mean_absolute_error(
    cold_user_recs['rating'],
    cold_user_recs['avg_rating']
)
print(round(rmse, 2), round(mae, 2)) 

0.78 0.62


Доп. код

In [19]:
# посчитаем покрытие холодных пользователей рекомендациями

cold_users_hit_ratio = (
    cold_users_events_with_recs
    .groupby("user_id")
    .agg(hits=("avg_rating", lambda x: (~x.isnull()).mean()))
)

print(f"Доля пользователей без релевантных рекомендаций: {(cold_users_hit_ratio == 0).mean().iat[0]:.2f}")
print(f"Среднее покрытие пользователей: {cold_users_hit_ratio[cold_users_hit_ratio != 0].mean().iat[0]:.2f}") 

Доля пользователей без релевантных рекомендаций: 0.59
Среднее покрытие пользователей: 0.44


# === Знакомство: первые персональные рекомендации

Задание 0 - степень разреженнности матрицы

In [20]:
# Кол-во книг(айтемов) и пользователей
(items['book_id'].nunique(), events['user_id'].nunique())

(43312, 430585)

In [21]:
1 - len(events) / (items['book_id'].nunique() * events['user_id'].nunique())

0.9993698979831817

Реализация SVD-алгоритма

In [22]:
from surprise import Dataset, Reader
from surprise import SVD

# используем Reader из библиотеки surprise для преобразования событий (events)
# в формат, необходимый surprise
reader = Reader(rating_scale=(1, 5))
surprise_train_set = Dataset.load_from_df(
    events_train[['user_id', 'book_id', 'rating']],
    reader
)
surprise_train_set = surprise_train_set.build_full_trainset()

# инициализируем модель
svd_model = SVD(n_factors=100, random_state=0)

# обучаем модель
svd_model.fit(surprise_train_set)

In [23]:
surprise_test_set = list(
    events_test[['user_id', 'book_id', 'rating']].itertuples(index=False)
)

# получаем рекомендации для тестовой выборки
svd_predictions = svd_model.test(surprise_test_set) 

In [24]:
# Расчет метрик встроенными методами библиотеки surprise
from surprise import accuracy

rmse = accuracy.rmse(svd_predictions)
mae = accuracy.mae(svd_predictions)
                     
print(rmse, mae) 

RMSE: 0.8289
MAE:  0.6474
0.8288711689059135 0.647437483750257


Для генерации случайных рекомендаций библиотека surprise предлагает класс NormalPredictor, который выдаёт случайные рейтинги из нормального распределения.

In [25]:
from surprise import NormalPredictor

# инициализируем состояние генератора, это необходимо для получения
# одной и той же последовательности случайных чисел, только в учебных целях
np.random.seed(0)

random_model = NormalPredictor()

random_model.fit(surprise_train_set)
random_predictions = random_model.test(surprise_test_set) 

In [26]:
# Оценка метрик для случайных рейтингов
mae_random = accuracy.mae(random_predictions)

MAE:  1.0018


На сколько процентов MAE для случайных рекомендаций от NormalPredictor выше значения MAE от SVD? Ответ округлите до десятых.

In [27]:
(mae_random - mae) / mae * 100

54.72886771309964

### Факультатив

Факультативное задание
Задание. Удалите из events события для редких айтемов — таких, с которыми взаимодействовало менее N пользователей. Возьмите небольшое N, например 2–3 пользователя. Получите рекомендации, посчитайте метрики, оцените, как они изменились. Подумайте, с чем могут быть связаны такие изменения.

Повторяем SVD предсказания для нередких айтемов

## Получение рекомендаций

In [28]:
def get_recommendations_svd(
        user_id,
        # all_items,
        events,
        model,
        include_seen=True,
        n=5
    ):
    """ возвращает n рекомендаций для user_id """

    # получим множество идентификаторов всех книг
    all_items = set(events['book_id'].unique())

    # учитываем флаг, стоит ли уже прочитанные книги включать в рекомендации
    if include_seen:
        items_to_predict = list(all_items)
    else:
        # получим список книг, которые пользователь уже прочитал ("видел")
        seen_items = set(events[events['user_id'] == user_id]['item_id']
                         .unique())
        
        # книги, которые пользователь ещё не читал
        # только их и будем включать в рекомендации
        items_to_predict = list(all_items - seen_items)
    
    # получаем скоры для списка книг, т. е. рекомендации
    predictions = [
        model.predict(user_id, item_id) for item_id in items_to_predict
    ]
    
    # сортируем рекомендации по убыванию скора и берём только n первых
    recommendations = sorted(predictions, key=lambda x: x.est, reverse=True)[:n]
    
    return pd.DataFrame([(pred.iid, pred.est) for pred in recommendations], columns=["book_id", "score"]) 

In [29]:
get_recommendations_svd(1296647, events_train, svd_model)

,book_id,score
0,24812,5.000000
1,7864312,4.981188
2,25793186,4.912001
3,12174312,4.898052
4,13208,4.894869


Дополнительная проверка качества рекомендаций

In [30]:
# выберем произвольного пользователя из тренировочной выборки ("прошлого")
user_id = events_train['user_id'].sample().iat[0]

print(f"user_id: {user_id}")

print("История (последние события, recent)")
user_history = (
    events_train
    .query("user_id == @user_id")
    .merge(items.set_index("book_id")[["author", "title", "genre_and_votes"]], on="book_id")
)
user_history_to_print = user_history[["author", "title", "started_at", "read_at", "rating", "genre_and_votes"]].tail(10)
display(user_history_to_print)

print("Рекомендации")
user_recommendations = get_recommendations_svd(user_id, events_train, svd_model)
user_recommendations = user_recommendations.merge(items[["book_id", "author", "title", "genre_and_votes"]], on="book_id")
display(user_recommendations)

user_id: 1169023
История (последние события, recent)


,author,title,started_at,read_at,rating,genre_and_votes
68,Veronica Roth,"Divergent (Divergent, #1)",2014-06-02,2014-06-04,4,"{'Young Adult': 20260, 'Science Fiction-Dystop..."
69,"Gillian Flynn, В. Русанов",Gone Girl,2014-05-27,2014-05-29,5,"{'Fiction': 11773, 'Mystery': 9965, 'Thriller'..."
70,Kathy Reichs,"Death du Jour (Temperance Brennan, #2)",2014-05-24,2014-05-27,4,"{'Mystery': 1206, 'Mystery-Crime': 579, 'Ficti..."
71,Chelsea Cain,"Heartsick (Archie Sheridan & Gretchen Lowell, #1)",2014-05-22,2014-05-22,5,"{'Mystery': 832, 'Thriller': 653, 'Fiction': 4..."
72,"Jussi Adler-Olsen, Lisa Hartford","The Keeper of Lost Causes (Department Q, #1)",2014-05-30,2014-06-02,3,"{'Mystery': 1225, 'Mystery-Crime': 627, 'Ficti..."
73,Gillian Flynn,Dark Places,2014-05-17,2014-05-22,4,"{'Mystery': 4534, 'Fiction': 4055, 'Thriller':..."
74,Audrey Niffenegger,Her Fearful Symmetry,2014-05-05,2014-05-08,2,"{'Fiction': 1984, 'Fantasy': 674, 'Fantasy-Par..."
75,Kathy Reichs,"Déjà Dead (Temperance Brennan, #1)",2014-05-13,2014-05-17,4,"{'Mystery': 2141, 'Fiction': 904, 'Mystery-Cri..."
76,Carolyn Parkhurst,The Dogs of Babel,2014-05-09,2014-05-10,5,"{'Fiction': 522, 'Mystery': 102, 'Animals': 77..."
77,George R.R. Martin,"A Dance with Dragons (A Song of Ice and Fire, #5)",2014-05-04,2014-05-04,5,"{'Fantasy': 22247, 'Fiction': 4512, 'Fantasy-E..."


Рекомендации


,book_id,score,author,title,genre_and_votes
0,2199,5,Doris Kearns Goodwin,Team of Rivals: The Political Genius of Abraha...,"{'History': 4174, 'Nonfiction': 2127, 'Biograp..."
1,16255632,5,"David Gaider, Ben Gelinas, Mike Laidlaw, Dave ...",Dragon Age: The World of Thedas Volume 1,"{'Fantasy': 134, 'Games-Video Games': 28, 'Art..."
2,2363958,5,João Guimarães Rosa,Grande Sertão: Veredas,"{'Fiction': 85, 'Classics': 69, 'Cultural-Braz..."
3,22552026,5,Jason Reynolds,Long Way Down,"{'Young Adult': 1871, 'Poetry': 1737, 'Contemp..."
4,29237211,5,"Brian K. Vaughan, Fiona Staples","Saga, Vol. 7 (Saga, #7)","{'Sequential Art-Graphic Novels': 2539, 'Seque..."


# === Базовые подходы: коллаборативная фильтрация

Код для перекодировки идентификаторов.

In [31]:
import scipy
import sklearn.preprocessing

# перекодируем идентификаторы пользователей: 
# из имеющихся в последовательность 0, 1, 2, ...
user_encoder = sklearn.preprocessing.LabelEncoder()
user_encoder.fit(events["user_id"])
events_train["user_id_enc"] = user_encoder.transform(events_train["user_id"])
events_test["user_id_enc"] = user_encoder.transform(events_test["user_id"])

# перекодируем идентификаторы объектов: 
# из имеющихся в последовательность 0, 1, 2, ...
item_encoder = sklearn.preprocessing.LabelEncoder()
item_encoder.fit(items["book_id"])
items["item_id_enc"] = item_encoder.transform(items["book_id"])
events_train["item_id_enc"] = item_encoder.transform(events_train["book_id"])
events_test["item_id_enc"] = item_encoder.transform(events_test["book_id"])

/tmp/ipykernel_1794/196496076.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_train["user_id_enc"] = user_encoder.transform(events_train["user_id"])
/tmp/ipykernel_1794/196496076.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_test["user_id_enc"] = user_encoder.transform(events_test["user_id"])
/tmp/ipykernel_1794/196496076.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the 

In [32]:
events_train['user_id_enc'].nunique(), events_train['item_id_enc'].nunique()

(428220, 41474)

Вычислите размер матрицы user_item_matrix_train, как если бы она хранила все свои элементы, включая пропуски, и для каждого элемента использовался бы один байт. Ответ приведите в виде целого числа гигабайтов, отбросив дробную часть.

In [33]:
(
    events_train['user_id_enc'].nunique() 
    * events_train['item_id_enc'].nunique()
    / (1024 ** 3)
)

16.54028546065092

In [34]:
# создаём sparse-матрицу формата CSR 
user_item_matrix_train = scipy.sparse.csr_matrix(
    (
        events_train["rating"],
        (events_train['user_id_enc'], events_train['item_id_enc'])
    ),
    dtype=np.int8
) 

In [35]:
from implicit.als import AlternatingLeastSquares
import os

os.environ['OPENBLAS_NUM_THREADS'] = '1'

als_model = AlternatingLeastSquares(
    factors=50,
    iterations=50,
    regularization=0.05,
    random_state=0
)
als_model.fit(user_item_matrix_train) 

/home/mle-user/mle_projects/mle-recsys-start/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/mle-user/mle_projects/mle-recsys-start/venv/lib/python3.10/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 4 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()
100%|██████████| 50/50 [02:43<00:00,  3.28s/it]


In [36]:
def get_recommendations_als(user_item_matrix, model, user_id, user_encoder, item_encoder, include_seen=True, n=5):
    """
    Возвращает отранжированные рекомендации для заданного пользователя
    """
    user_id_enc = user_encoder.transform([user_id])[0]
    recommendations = model.recommend(
         user_id_enc, 
         user_item_matrix[user_id_enc], 
         filter_already_liked_items=not include_seen,
         N=n)
    recommendations = pd.DataFrame({"item_id_enc": recommendations[0], "score": recommendations[1]})
    recommendations["item_id"] = item_encoder.inverse_transform(recommendations["item_id_enc"])
    
    return recommendations 

Факультативное задание
Используя get_recommendations_als, напишите код, который позволит для случайного пользователя просмотреть рекомендации в удобном формате: 
история с именами авторов и названием книг,
рекомендации с именами авторов и названием книг, seen-признаком (взаимодействовал ли уже пользователь с рекомендованной книгой).
Проанализируйте, релевантны ли рекомендации имеющейся истории.

In [37]:
# получаем список всех возможных user_id (перекодированных)
user_ids_encoded = range(len(user_encoder.classes_))

# получаем рекомендации для всех пользователей
als_recommendations = als_model.recommend(
    user_ids_encoded, 
    user_item_matrix_train[user_ids_encoded], 
    filter_already_liked_items=False,
    N=100
)

In [38]:
# преобразуем полученные рекомендации в табличный формат
item_ids_enc = als_recommendations[0]
als_scores = als_recommendations[1]

als_recommendations = pd.DataFrame({
    "user_id_enc": user_ids_encoded,
    "item_id_enc": item_ids_enc.tolist(), 
    "score": als_scores.tolist()})
als_recommendations = als_recommendations.explode(["item_id_enc", "score"], ignore_index=True)

# приводим типы данных
als_recommendations["item_id_enc"] = als_recommendations["item_id_enc"].astype("int")
als_recommendations["score"] = als_recommendations["score"].astype("float")

# получаем изначальные идентификаторы
als_recommendations["user_id"] = user_encoder.inverse_transform(als_recommendations["user_id_enc"])
als_recommendations["item_id"] = item_encoder.inverse_transform(als_recommendations["item_id_enc"])
als_recommendations = als_recommendations.drop(columns=["user_id_enc", "item_id_enc"]) 

In [39]:
als_recommendations = als_recommendations[["user_id", "item_id", "score"]]
als_recommendations.to_parquet("goodsread/als_recommendations.parquet")

Для удобства оценки добавим в датафрейм с рекомендациями истинные оценки из тестовой выборки:

In [40]:
als_recommendations = (
    als_recommendations
    .merge(
        events_test[["user_id", "book_id", "rating"]]
        .rename(columns={
            "rating": "rating_test",
            "book_id": 'item_id'
        }), 
        on=["user_id", "item_id"], how="left")
)

In [41]:
import sklearn.metrics

def compute_ndcg(rating: pd.Series, score: pd.Series, k):

    """ подсчёт ndcg
    rating: истинные оценки
    score: оценки модели
    k: количество айтемов (по убыванию score) для оценки, остальные - отбрасываются
    """
    
    # если кол-во объектов меньше 2, то NDCG - не определена
    if len(rating) < 2:
        return np.nan

    ndcg = sklearn.metrics.ndcg_score(np.asarray([rating.to_numpy()]), np.asarray([score.to_numpy()]), k=k)

    return ndcg

Умея считать NDCG для одного пользователя, посчитаем данную метрику, например, для 𝑘=5 для всех пользователей из тестовой выборки. В результате каждому пользователю будет соответствовать одно значение NDCG@5.

In [42]:
rating_test_idx = ~als_recommendations["rating_test"].isnull()
ndcg_at_5_scores = als_recommendations[rating_test_idx].groupby("user_id").apply(lambda x: compute_ndcg(x["rating_test"], x["score"], k=5))

In [43]:
print(ndcg_at_5_scores.mean())

0.975946709792109


Факультативное задание
Оцените, для какой доли пользователей удалось посчитать метрику NDCG.

In [44]:
sum(ndcg_at_5_scores > 0) / len(ndcg_at_5_scores)

0.35807624389737197

Используя метод  similar_items, получите и оцените рекомендации для нескольких айтемов. Проанализируйте адекватность результатов.

Факультативное задание
Используя метод  similar_items, получите и оцените рекомендации для нескольких айтемов. Проанализируйте адекватность результатов.

# === Базовые подходы: контентные рекомендации

In [45]:
items["genre_and_votes"] = items["genre_and_votes"].apply(eval) 

Задание 1 из 4
Теперь составьте список жанров с долями голосов по ним в genres.

In [46]:
def get_genres(items):
    """ 
    извлекает список жанров по всем книгам, 
    подсчитывает долю голосов по каждому их них
    """
    
    genres_counter = {}
    
    for k, v, in items.iterrows():
        genre_and_votes = v['genre_and_votes']
        if genre_and_votes is None or not isinstance(genre_and_votes, dict):
            continue
        for genre, votes in genre_and_votes.items():
            # увеличиваем счётчик жанров
            try:
                genres_counter[genre] += votes
            except KeyError:
                genres_counter[genre] = votes

    genres = pd.Series(genres_counter, name="votes")
    genres = genres.to_frame()
    genres = genres.reset_index().rename(columns={"index": "name"})
    genres.index.name = "genre_id"
    
    return genres

In [47]:
   
genres = get_genres(items) 

In [48]:
genres["score"] = genres["votes"] / genres["votes"].sum()
genres.sort_values(by="score", ascending=False).head(10)

,name,votes,score
genre_id,,,
25,Fantasy,6850115,0.149498
1,Fiction,6406698,0.139821
38,Classics,3415071,0.074531
18,Young Adult,3297027,0.071955
34,Romance,2422690,0.052873
5,Nonfiction,1737798,0.037926
16,Historical-Historical Fiction,1531489,0.033423
20,Mystery,1371370,0.029929
24,Science Fiction,1218997,0.026604


Задание 2 из 4 Функция в коде ниже строит матрицу вида «книга-жанр». Изучите её. Подумайте, что будет соответствовать столбцам матрицы

In [49]:
def get_item2genre_matrix(genres, items):

    genre_names_to_id = genres.reset_index().set_index("name")["genre_id"].to_dict()
    
    # list to build CSR matrix
    genres_csr_data = []
    genres_csr_row_idx = []
    genres_csr_col_idx = []
    
    for item_idx, (k, v) in enumerate(items.iterrows()):
        if v["genre_and_votes"] is None:
            continue
        for genre_name, votes in v["genre_and_votes"].items():
            genre_idx = genre_names_to_id[genre_name]
            genres_csr_data.append(int(votes))
            genres_csr_row_idx.append(item_idx)
            genres_csr_col_idx.append(genre_idx)

    genres_csr = scipy.sparse.csr_matrix((genres_csr_data, (genres_csr_row_idx, genres_csr_col_idx)), shape=(len(items), len(genres)))
    # нормализуем, чтобы сумма оценок принадлежности к жанру была равна 1
    genres_csr = sklearn.preprocessing.normalize(genres_csr, norm='l1', axis=1)
    
    return genres_csr 

In [50]:
items = items.sort_values(by="item_id_enc")
all_items_genres_csr = get_item2genre_matrix(genres, items)

Аналогичным образом получим матрицу с весами по жанрам для какого-нибудь пользователя, например, для пользователя с идентификатором 1000010. 
Задание 3 из 4 Дополните и выполните код ниже, чтобы получить описанную матрицу. 

In [51]:
user_id = 1000010
user_events = events_train.query("user_id == @user_id")[["book_id", "rating"]]
user_items = items[items["book_id"].isin(user_events["book_id"])]

user_items_genres_csr = get_item2genre_matrix(genres, user_items)
user_items_genres_csr

<22x815 sparse matrix of type '<class 'numpy.float64'>'
	with 149 stored elements in Compressed Sparse Row format>

In [52]:
user_ratings = user_events["rating"].to_numpy() / 5
user_ratings = np.expand_dims(user_ratings, axis=1)

user_items_genres_weighted = user_items_genres_csr.multiply(user_ratings)

user_items_genres_weighted.shape

(22, 815)

In [53]:
# вычислим склонность пользователя к жанрам как среднее взвешенное значение популяции на его оценки книг.
# преобразуем пользовательские оценки из списка в вектор-столбец

user_ratings = user_events["rating"].to_numpy() / 5
user_ratings = np.expand_dims(user_ratings, axis=1)

user_items_genres_weighted = user_items_genres_csr.multiply(user_ratings)

user_genres_scores = np.asarray(user_items_genres_weighted.mean(axis=0))

In [54]:
user_genres_scores.shape

(1, 815)

Задание 4 из 4 Получите наиболее релевантные рекомендации для пользователя. Дополните код так, чтобы переменная top_k_indices заполнялась индексами соответствующих книг. Для этого удобно использовать np.argsort от similarity_scores, подсчитанной для всех книг.

In [55]:
from sklearn.metrics.pairwise import cosine_similarity

# вычисляем сходство между вектором пользователя и векторами по книгам
similarity_scores = cosine_similarity(all_items_genres_csr, user_genres_scores)

# преобразуем в одномерный массив
similarity_scores = similarity_scores.flatten()

# получаем индексы top-k (по убыванию значений), по сути, индексы книг (encoded)
k = 5
top_k_indices = np.argsort(similarity_scores)[:k]

In [56]:
top_k_indices

array([38862, 38838,  9716, 18969, 18925])

In [57]:
selected_items = items[items["item_id_enc"].isin(top_k_indices)]

with pd.option_context("max_colwidth", 100):
   display(selected_items[["author", "title", "genre_and_votes"]])

,author,title,genre_and_votes
1673075,Mo Willems,"Today I Will Fly! (Elephant & Piggie, #1)","{'Childrens-Picture Books': 409, 'Childrens': 199}"
1523576,Lester Sumrall,Gifts and Ministries of the Holy Spirit,"{'Christian': 6, 'Religion-Theology': 3, 'Christian-Christian Non Fiction': 2, 'Spirituality': 2}"
1422146,Nicholas Davies,Diana the Killing of a Princess,None
515584,Nayyirah Waheed,Nejma,"{'Poetry': 929, 'Feminism': 31}"
956516,"Tina Marie Kaht, Valeria Avantario",Grandparents' Day,{'Childrens': 2}


Факультативное задание
Получите по алгоритму выше рекомендации для нескольких пользователей, просмотрите их на экране. Подумайте, насколько релевантны и интересны полученные рекомендации пользователям.
Попробуйте использовать другую меру сходства для получения рекомендаций, например, евклидово расстояние. Проанализируйте, отличаются ли рекомендации от предыдущих. Подумайте почему.
Задайте собственные предпочтения для наиболее популярных жанров. Посмотрите рекомендации для себя. Прочитали ли бы вы рекомендованные книги?

# === Базовые подходы: валидация

Посчитаем recall и precision для ALS-рекомендаций (als_recommendations). Для этого события в тестовой выборке и рекомендации для одних и тех же пользователей разметим признаками:
gt (ground truth): объект есть в тестовой выборке;
pr (predicted): объект есть в рекомендациях.
Теперь разметим признаки бинарной классификации:
TP: объект есть и в тестовой выборке, и в рекомендациях (истинная рекомендация),
FP: объекта нет в тестовой выборке, но он есть в рекомендациях (ложноположительная рекомендация),
FN: объект есть в тестовой выборке, но его нет в рекомендациях (ложноотрицательная рекомендация)

In [58]:
def process_events_recs_for_binary_metrics(events_train, events_test, recs, top_k=None):

    """
    размечает пары <user_id, item_id> для общего множества пользователей признаками
    - gt (ground truth)
    - pr (prediction)
    top_k: расчёт ведётся только для top k-рекомендаций
    """

    events_test["gt"] = True
    common_users = set(events_test["user_id"]) & set(recs["user_id"])

    print(f"Common users: {len(common_users)}")
    
    events_for_common_users = events_test[events_test["user_id"].isin(common_users)].rename(columns={'book_id': 'item_id'}).copy()
    recs_for_common_users = recs[recs["user_id"].isin(common_users)].copy()

    recs_for_common_users = recs_for_common_users.sort_values(["user_id", "score"], ascending=[True, False])

    # оставляет только те item_id, которые были в events_train, 
    # т. к. модель не имела никакой возможности давать рекомендации для новых айтемов
    events_for_common_users = events_for_common_users[events_for_common_users["item_id"].isin(events_train["book_id"].unique())]

    if top_k is not None:
        recs_for_common_users = recs_for_common_users.groupby("user_id").head(top_k)
    
    events_recs_common = events_for_common_users[["user_id", "item_id", "gt"]].merge(
        recs_for_common_users[["user_id", "item_id", "score"]], 
        on=["user_id", "item_id"], how="outer")    

    events_recs_common["gt"] = events_recs_common["gt"].fillna(False)
    events_recs_common["pr"] = ~events_recs_common["score"].isnull()
    
    events_recs_common["tp"] = events_recs_common["gt"] & events_recs_common["pr"]
    events_recs_common["fp"] = ~events_recs_common["gt"] & events_recs_common["pr"]
    events_recs_common["fn"] = events_recs_common["gt"] & ~events_recs_common["pr"]

    return events_recs_common 

Обработаем ALS-рекомендации для подсчёта метрик для 5 лучших рекомендаций:

In [59]:
events_recs_for_binary_metrics = process_events_recs_for_binary_metrics(
  events_train,
    events_test, 
    als_recommendations, 
    top_k=5) 

/tmp/ipykernel_1794/575309485.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_test["gt"] = True


Common users: 123223


Дополните код функции compute_cls_metrics для расчёта recall. Получите значения метрик precision@5, recall@5.

In [60]:
def compute_cls_metrics(events_recs_for_binary_metric):
    
    groupper = events_recs_for_binary_metric.groupby("user_id")

    # precision = tp / (tp + fp)
    precision = groupper["tp"].sum()/(groupper["tp"].sum()+groupper["fp"].sum())
    precision = precision.fillna(0).mean()
    
    # recall = tp / (tp + fn)
    recall = groupper["tp"].sum()/(groupper["tp"].sum()+groupper["fn"].sum())
    recall = recall.fillna(0).mean()

    return precision, recall 

In [61]:
compute_cls_metrics(events_recs_for_binary_metrics)

(0.007581376853347184, 0.014121568795222568)

Факультативное задание
Посчитайте метрики precision@10, recall@10. Сравните их значения со значениями для precision@5, recall@10. Подумайте о причинах таких отличий.

# === Двухстадийный подход: метрики

In [62]:
# ## Загружаем сохраненный als_reccomendations
# als_recommendations = pd.read_parquet('goodsread/als_recommendations.parquet')

Для рекомендаций, сохранённых в переменной als_recommendations, посчитайте покрытие по объектам согласно формуле выше. При этом используйте весь топ-100 рекомендаций.

In [63]:
# расчёт покрытия по объектам
cov_items = als_recommendations['item_id'].nunique() / len(items)
print(f"{cov_items:.2f}")

0.09


Посчитайте среднее Novelty@5 для als_recommendations. Для этого: 
разметьте каждую рекомендацию в als_recommendations булевым признаком read (False — пользователь не читал книгу, True — пользователь читал книгу), используя events_train,
посчитайте Novelty@5 для каждого пользователя,
посчитайте среднеарифметическое для полученных значений Novelty@5

In [64]:
events_train = events_train.rename(columns={'book_id': 'item_id'})
events_test = events_test.rename(columns={'book_id': 'item_id'})
events_train.head(1)

,user_id,item_id,started_at,read_at,is_read,rating,is_reviewed,started_at_month,user_id_enc,item_id_enc
0,1229132,22034,2015-07-12,2015-07-17,True,5,False,2015-07-01,229132,2460


In [65]:
als_recommendations.head(1)

,user_id,item_id,score,rating_test
0,1000000,3,0.990941,NaN


In [66]:
# разметим каждую рекомендацию признаком read
events_train["read"] = True
als_recommendations = als_recommendations.merge(
    events_train[['read', 'user_id', 'item_id']], 
    on=["user_id", "item_id"],
    how="left"
)
als_recommendations["read"] = (
    als_recommendations["read"]
    .fillna(False)
    .astype("bool")
)

# проставим ранги
als_recommendations = als_recommendations.sort_values(
    by=['user_id', 'score'],
    ascending=[True, False]
)
als_recommendations["rank"] = (
    als_recommendations
    .groupby("user_id")
    .cumcount()
 ) + 1

# посчитаем novelty по пользователям
novelty_5 = (1-als_recommendations.query("rank <= 5").groupby("user_id")["read"].mean())

# посчитаем средний novelty
novelty_mean = novelty_5.mean()

In [67]:
novelty_mean

0.607333279143491

# === Двухстадийный подход: модель

Задание 1 из 6
Используем отложенную тестовую часть данных — назовём её events_test — для получения двух новых частей данных:
одна, составляющая первые 45 дней, будет использоваться для таргетов,
другая, состоящая из 45 последних дней, будет новой тестовой выборкой.
Завершите код так, чтобы в events_labels оказалась первая часть данных, а в events_test_2 — вторая.

In [68]:
# задаём точку разбиения
split_date_for_labels = pd.to_datetime("2017-09-15").date()

split_date_for_labels_idx = events_test["started_at"] < split_date_for_labels
events_labels = events_test[split_date_for_labels_idx].copy()
events_test_2 = events_test[split_date_for_labels_idx].copy() 

In [69]:
events_labels['user_id'].nunique()

99849

Подготовим список кандидатов для обучения ранжирующей модели. В качестве кандидатогенераторов возьмём ALS и контентную модель на основе жанровых предпочтений, известных нам из прошлых уроков. Рекомендации от них были заранее подготовлены и сохранены в файлах als_recommendations.parquet и content_recommendations.parquet в директории candidates/training. Подготовка заключается в объединении списков рекомендаций по совпадению user_id, item_id.     
Задание 2 из 6     
Объедините имеющихся кандидатов по совпадению user_id, item_id в один список.

In [70]:
# загружаем рекомендации от двух базовых генераторов
als_recommendations = pd.read_parquet("candidates/training/als_recommendations.parquet")
content_recommendations = pd.read_parquet("candidates/training/content_recommendations.parquet")

In [71]:
candidates = pd.merge(
    als_recommendations[["user_id", "item_id", "score"]].rename(columns={"score": "als_score"}),
    content_recommendations[["user_id", "item_id", "score"]].rename(columns={"score": "cnt_score"}),
    on=["user_id", "item_id"],
    how="outer"
)

In [72]:
len(candidates)

82993094

In [73]:
candidates.head(1)

,user_id,item_id,als_score,cnt_score
0,1000000,3,0.972557,0.920225


Задание 3 из 6   
Дополните код ниже.
В candidates добавьте колонку target со значениями:
1 для тех item_id, которые пользователь прочитал (положительный пример).
0 — для всех остальных (негативный пример).
В candidates_for_train отберите все положительные примеры, а также не менее четырёх негативных примеров для каждого пользователя в положительных примерах.

In [74]:
# добавляем таргет к кандидатам со значением:
# — 1 для тех item_id, которые пользователь прочитал
# — 0, для всех остальных 

events_labels["target"] = 1
candidates = candidates.merge(
    events_labels[["user_id", "item_id", "target"]], 
    on=["user_id", "item_id"],
    how='left'
)
candidates["target"] = candidates["target"].fillna(0).astype("int")

# в кандидатах оставляем только тех пользователей, у которых есть хотя бы один положительный таргет
candidates_to_sample = candidates.groupby("user_id").filter(lambda x: x["target"].sum() > 0)

# для каждого пользователя оставляем только 4 негативных примера
negatives_per_user = 4
candidates_for_train = pd.concat([
    candidates_to_sample.query("target == 1"),
    candidates_to_sample.query("target == 0") \
        .groupby("user_id") \
        .apply(lambda x: x.sample(negatives_per_user, random_state=0))
    ]) 

In [75]:
candidates_for_train.shape

(213708, 5)

In [76]:
candidates_for_train.head(1)

,user_id,item_id,als_score,cnt_score,target
615,1000006,29868610,0.286715,NaN,1


Выше мы подготовили все нужные артефакты для обучения модели ранжирования. Выполните код, чтобы обучить модель.

In [77]:
from catboost import CatBoostClassifier, Pool

# задаём имена колонок признаков и таргета
features = ['als_score', 'cnt_score']
target = 'target'

# Create the Pool object
train_data = Pool(
    data=candidates_for_train[features], 
    label=candidates_for_train[target])

# инициализируем модель CatBoostClassifier
cb_model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.1,
    depth=6,
    loss_function='Logloss',
    verbose=100,
    random_seed=0
)

# тренируем модель
cb_model.fit(train_data)

0:	learn: 0.6526057	total: 72.7ms	remaining: 1m 12s
100:	learn: 0.5118959	total: 1.76s	remaining: 15.7s
200:	learn: 0.5111710	total: 3.52s	remaining: 14s
300:	learn: 0.5105208	total: 5.27s	remaining: 12.2s
400:	learn: 0.5100174	total: 7.02s	remaining: 10.5s
500:	learn: 0.5095747	total: 8.78s	remaining: 8.75s
600:	learn: 0.5091600	total: 10.6s	remaining: 7.01s
700:	learn: 0.5087803	total: 12.3s	remaining: 5.26s
800:	learn: 0.5084220	total: 14.1s	remaining: 3.5s
900:	learn: 0.5080930	total: 15.9s	remaining: 1.74s
999:	learn: 0.5078081	total: 17.6s	remaining: 0us


: 

Подготовка кандидатов для рекомендаций
Представим, что натренированная модель используется только некоторое время спустя, когда уже появились новые рекомендации (кандидаты) от базовых генераторов, обученных на объединении событий из events_train и events_label. Иными словами, когда события из events_label уже стали частью тренировочного набора данных. Эти новые рекомендации были заранее подготовлены и сохранены в файлах als_recommendations.parquet и content_recommendations.parquet в директории candidates/inference. Используем их для составления нового списка кандидатов candidates_to_rank, который понадобится готовой ранжирующей модели. 
Задание 4 из 6
Дополните код ниже так, чтобы в candidates_to_rank попали кандидаты от обоих базовых генераторов подобно тому, как это было сделано для фазы тренировки выше.

In [78]:
# загружаем рекомендации от двух базовых генераторов
als_recommendations_2 = pd.read_parquet("candidates/inference/als_recommendations.parquet")
content_recommendations_2 = pd.read_parquet("candidates/inference/content_recommendations.parquet")

candidates_to_rank = pd.merge(
    als_recommendations_2[["user_id", "item_id", "score"]].rename(columns={"score": "als_score"}),
    content_recommendations_2[["user_id", "item_id", "score"]].rename(columns={"score": "cnt_score"}),
    on=["user_id", "item_id"],
    how="outer"
)

# оставляем только тех пользователей, что есть в тестовой выборке, для экономии ресурсов
candidates_to_rank = candidates_to_rank[candidates_to_rank["user_id"].isin(events_test["user_id"].drop_duplicates())]
print(len(candidates_to_rank))

Применим обученную ранжирующую модель к кандидатам для рекомендаций. Таргет уже не нужен, поскольку мы будем применять модель в режиме инференса.
Задание 5 из 6
Дополните код для того, чтобы вызвать модель и получить для каждого пользователя топ-100 рекомендаций — значение rank нужно выставить не более ста.

In [ ]:
inference_data = Pool(data=candidates_to_rank[features])
predictions = cb_model.predict_proba(inference_data)

candidates_to_rank["cb_score"] = predictions[:, 1]

# для каждого пользователя проставляем rank, начиная с 1 — это максимальный cb_score
candidates_to_rank = candidates_to_rank.sort_values(["user_id", "cb_score"], ascending=[True, False])
candidates_to_rank["rank"] = (
    candidates_to_rank
    .groupby("user_id")
    .cumcount()
 ) + 1
max_recommendations_per_user = 100
final_recommendations = candidates_to_rank.query('rank <= @max_recommendations_per_user')

In [ ]:
len(final_recommendations)

Задание 6 из 6
Посчитайте метрики recall и precision.
Используйте полученные рекомендации final_recommendations, отложенную тестовую выборку events_test_2, созданные в уроке «Валидация» предыдущей темы.
А также функции process_events_recs_for_binary_metrics и compute_cls_metrics.

In [ ]:
events_inference = pd.concat([events_train, events_labels])

cb_events_recs_for_binary_metrics_5 = process_events_recs_for_binary_metrics(
    events_inference.rename(columns={'item_id': 'book_id'}),
    events_test_2,
    final_recommendations.rename(columns={"cb_score": "score"}), 
    top_k=5
)

cb_precision_5, cb_recall_5 = compute_cls_metrics(cb_events_recs_for_binary_metrics_5)

print(f"precision: {cb_precision_5:.3f}, recall: {cb_recall_5:.3f}") 

# === Двухстадийный подход: построение признаков

Посчитаем новый признак — «возраст» книги на основе года публикации. Назовём его age. 
Используя обновлённый справочник объектов items, добавьте признаки возраста age и средней популярности average_rating к кандидатам для тренировки модели candidates_for_train и к кандидатам для ранжирования candidates_to_rank.

In [ ]:
items.head(1)

In [ ]:
items["age"] = 2018-items["publication_year"]
invalid_age_idx = items["age"] < 0
items.loc[invalid_age_idx, "age"] = np.nan
items["age"] = items["age"].astype("float")

candidates_for_train = candidates_for_train.merge(
    items[['book_id', "age"]].rename(columns={'book_id': 'item_id'}),
    on=['item_id'],
    how='left'
)
candidates_to_rank = candidates_to_rank.merge(
    items[['book_id', "age"]].rename(columns={'book_id': 'item_id'}),
    on=['item_id'],
    how='left'
)

In [ ]:
candidates_to_rank['age'].median()

Используя события в events_train и events_inference, посчитайте и добавьте признаки пользователей к кандидатам в candidates_for_train и candidates_to_rank соответственно:
reading_years — длительность истории пользователя,
books_read — количество книг, прочитанных за всё время,
books_per_year — среднее количество прочитанных книг в год,
rating_avg — средняя оценка,
rating_std — дисперсия оценок.

In [ ]:
events.head(1)

In [ ]:
def get_user_features(events):
    """ считает пользовательские признаки """
    
    user_features = events.groupby("user_id").agg(
        reading_years=("started_at", lambda x: (x.max()-x.min()).days/365.25),
        books_read=('is_read', 'sum'),
        rating_avg=("rating", "mean"),
        rating_std=("rating", "std")
    )
    
    user_features["books_per_year"] = user_features["books_read"] / user_features["reading_years"]
    
    return user_features
    
user_features_for_train = get_user_features(events_train)
candidates_for_train = candidates_for_train.merge(user_features_for_train, on="user_id", how="left")
  
# оставим только тех пользователей, что есть в тесте, для экономии ресурсов
events_inference = pd.concat([events_train, events_labels])
events_inference = events_inference[events_inference["user_id"].isin(events_test["user_id"].drop_duplicates())]

user_features_for_ranking = get_user_features(events_inference)
candidates_to_rank = candidates_to_rank.merge(user_features_for_ranking, on="user_id", how="left")

In [ ]:
candidates_for_train['books_read'].median()

Используя истории events_train и events_inference, а также ранее полученные артефакты по жанрам книг — словарь жанров genres, оценки книг по жанрам all_items_genres_csr — добавьте парные признаки, по одному на каждый жанр, которые совместно показывают, какие жанры предпочитает пользователь. 
Жанровость в данном случае — численный коэффициент принадлежности книги к жанру. Например, если пользователь прочитал три книги, которые с весами 0.3, 0.2, 0.4 из  all_items_genres_csr относятся к Fantasy, то интерес пользователя к Fantasy составляет среднее этих трёх оценок — 0.3.
Для экономии ресурсов возьмём не все жанры, а десять наиболее популярных. Все остальные отметим как не вошедшие в топ и обозначим как others. 

In [ ]:
genres.head(1)

In [ ]:
# определяем индексы топ-10 жанров и всех остальных
genres_top_k = 10
genres_top_idx = genres.sort_values("votes", ascending=False).head(genres_top_k).index
genres_others_idx = list(set(genres.index) - set(genres_top_idx))

genres_top_columns = [f"genre_{id}" for id in genres_top_idx]
genres_others_column = "genre_others"
genre_columns = # ваш код здесь #

# составляем таблицу принадлежности книг к жанрам
item_genres = (
    pd.concat([
        # топ жанров
        # ваш код здесь #,
        # все остальные жанры
        pd.DataFrame(all_items_genres_csr[:, genres_others_idx].sum(axis=1), columns=[genres_others_column])
        ],
        axis=1)
    .reset_index()
    .rename(columns={"index": "item_id_enc"})
)

# объединяем информацию принадлежности книг к жанрам с основной информацией о книгах
items = items.merge(item_genres, on="item_id_enc", how="left")

def get_user_genres(events, items, item_genre_columns):
    user_genres = (
        events
        .merge(items[["item_id"] + item_genre_columns], on="item_id", how="left")
        .groupby("user_id")[item_genre_columns].mean()
    )
    return user_genres
    
user_genres_for_train = # ваш код здесь #
candidates_for_train = candidates_for_train.merge(user_genres_for_train, on="user_id", how="left")

user_genres_for_ranking = get_user_genres(events_inference, items, genre_columns)
candidates_to_rank = candidates_to_rank.merge(user_genres_for_ranking, on="user_id", how="left")